In [170]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 26 16:09:11 2017

@author: cbothore
"""

import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import pylab
import numpy as np
import pickle
from collections import Counter

def naive_method(graph, empty, attr):
    """   Predict the missing attribute with a simple but effective
    relational classifier. 
    
    The assumption is that two connected nodes are 
    likely to share the same attribute value. Here we chose the most frequently
    used attribute by the neighbors
    
    Parameters
    ----------
    graph : graph
       A networkx graph
    empty : list
       The nodes with empty attributes 
    attr : dict 
       A dict of attributes, either location, employer or college attributes. 
       key is a node, value is a list of attribute values.

    Returns
    -------
    predicted_values : dict 
       A dict of attributes, either location, employer or college attributes. 
       key is a node (from empty), value is a list of attribute values. Here 
       only 1 value in the list.
     """
    predicted_values={}
    for n in empty:
        nbrs_attr_values=[] 
        for nbr in graph.neighbors(n):
            if nbr in attr:
                for val in attr[nbr]:
                    nbrs_attr_values.append(val)
        predicted_values[n]=[]
        if nbrs_attr_values: # non empty list
            # count the number of occurrence each value and returns a dict
            cpt=Counter(nbrs_attr_values)
            # take the most represented attribute value among neighbors
            a,nb_occurrence=max(cpt.items(), key=lambda t: t[1])
            predicted_values[n].append(a)
    return predicted_values
    
def evaluation_accuracy(groundtruth, pred):
    """    Compute the accuracy of your model.

     The accuracy is the proportion of true results.

    Parameters
    ----------
    groundtruth :  : dict 
       A dict of attributes, either location, employer or college attributes. 
       key is a node, value is a list of attribute values.
    pred : dict 
       A dict of attributes, either location, employer or college attributes. 
       key is a node, value is a list of attribute values. 

    Returns
    -------
    out : float
       Accuracy.
    """
    true_positive_prediction=0   
    for p_key, p_value in pred.items():
        if p_key in groundtruth:
            # if prediction is no attribute values, e.g. [] and so is the groundtruth
            # May happen
            if not p_value and not groundtruth[p_key]:
                true_positive_prediction+=1
            # counts the number of good prediction for node p_key
            # here len(p_value)=1 but we could have tried to predict more values
            true_positive_prediction += len([c for c in p_value if c in groundtruth[p_key]])          
        # no else, should not happen: train and test datasets are consistent
    return true_positive_prediction*100/sum(len(v) for v in pred.values())

In [2]:
# load the graph
G = nx.read_gexf("mediumLinkedin.gexf")
print("Nb of users in our graph: %d" % len(G))

Nb of users in our graph: 811


In [215]:
# load the profiles. 3 files for each type of attribute
# Some nodes in G have no attributes
# Some nodes may have 1 attribute 'location'
# Some nodes may have 1 or more 'colleges' or 'employers', so we
# use dictionaries to store the attributes
college={}
location={}
employer={}
# The dictionaries are loaded as dictionaries from the disk (see pickle in Python doc)
with open('mediumCollege_60percent_of_empty_profile.pickle', 'rb') as handle:
    college = pickle.load(handle)
with open('mediumLocation_60percent_of_empty_profile.pickle', 'rb') as handle:
    location = pickle.load(handle)
with open('mediumEmployer_60percent_of_empty_profile.pickle', 'rb') as handle:
    employer = pickle.load(handle)

print("Nb of users with one or more attribute college: %d" % len(college))
print("Nb of users with one or more attribute location: %d" % len(location))
print("Nb of users with one or more attribute employer: %d" % len(employer))

Nb of users with one or more attribute college: 575
Nb of users with one or more attribute location: 840
Nb of users with one or more attribute employer: 742


In [208]:
# here are the empty nodes for whom your challenge is to find the profiles
empty_nodes=[]
with open('mediumRemovedNodes_60percent_of_empty_profile.pickle', 'rb') as handle:
    empty_nodes = pickle.load(handle)
print("Your mission, find attributes to %d users with empty profile" % len(empty_nodes))

Your mission, find attributes to 475 users with empty profile


In [217]:
# --------------------- Baseline method -------------------------------------#
# Try a naive method to predict attribute
# This will be a baseline method for you, i.e. you will compare your performance
# with this method
# Let's try with the attribute 'employer'
employer_predictions=premier_method(G, empty_nodes, location)
groundtruth_employer={}
with open('largeLocation.pickle', 'rb') as handle:
    groundtruth_employer = pickle.load(handle)
result=evaluation_accuracy(groundtruth_employer,employer_predictions)
print("%f%% of the predictions are true" % result)
print("Very poor result!!! Try to do better!!!!")

# --------------------- Now your turn -------------------------------------#
# Explore, implement your strategy to fill empty profiles of empty_nodes


# and compare with the ground truth (what you should have predicted)
# user precision and recall measures

30.947368% of the predictions are true
Very poor result!!! Try to do better!!!!


In [164]:
def initial_dict(graph, empty, attr):
    mydict = {}
    for n in empty: 
        i = 0
        for nb in graph.neighbors(n):
            if nb in attr:
                i = i+1
        mydict[n] = i
    return mydict

def get_max(mydict deja):
    m = 0
    for k,v in mydict.items():
        if k not in deja:
            if v > m: m = v
    return m

In [201]:
def premier_method(graph, empty, attr):
    
    predicted_values = {}
    mydict = initial_dict(graph, empty, attr)
        
    deja = []
    while(len(mydict) != len(deja)):
        mx = get_max(mydict, deja)
        for k,v in mydict.items():
            if k not in deja and v == mx:
                nbrs = []
                for nb in graph.neighbors(k):
                    if nb in attr: nbrs.extend(attr[nb])
                    if nb in predicted_values: nbrs.extend(predicted_values[nb])
                    if nb in mydict: mydict[nb] = mydict[nb]+1
                cnt = Counter(nbrs)
                predicted_values[k] = [max(cnt, key=lambda k: cnt[k])]
                deja.append(k)
                
    """
    # counts for how many neighbors per node the attribute is known
    
    
   
            
                    nbrs = []
                    for nb in graph.neighbors(k):
                        if nb in attr:
                            nbrs.append(attr[nb])
                            print(nbrs)
                    cnt = Counter(nbrs)
                    predicted_values[k] = (max(cnt, key=lambda k: cnt[k]))
                    print(k, predicted_values[k])
                    attr[k] = predicted_values[k]
                    deja.append(k)
    """
    return predicted_values

employer_predictions = premier_method(G, empty_nodes, location)

In [196]:
for i in G.neighbors('U24085'):
    print(i)

U2691
U24082


In [73]:
a = {'a':[1,2,3],'b':[1,2,3]}
b = []
b.extend(a['a'])
b

[1, 2, 3]

In [104]:
a = {'a':0,'b':1,'c':2,'d':3,'e':5,'f':6}

TypeError: 'builtin_function_or_method' object is not subscriptable

In [131]:
a
for [k, v] in a.items():
    print(v)

0
1
2
3
5
6


In [109]:
len(numbers)

1

In [110]:
i = 0

In [111]:
i++

SyntaxError: invalid syntax (<ipython-input-111-56f7e036d680>, line 1)